In [1]:
!nvidia-smi

Sat Aug  9 15:46:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40S                    On  |   00000000:33:00.0 Off |                    0 |
| N/A   30C    P8             33W /  350W |       1MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
# imports
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer 
from datasets import load_dataset
import pandas as pd 
import numpy as np
from huggingface_hub import login
import os

from tqdm import tqdm

In [3]:
import numpy as np
print(np.__file__)  # should show a proper path
print(np.__version__)

import pandas as pd
print(pd.__file__)  # should show a proper path
print(pd.__version__)

/rds/general/user/nk1924/home/LASR-probe-gen/.venv/lib/python3.9/site-packages/numpy/__init__.py
1.26.4
/rds/general/user/nk1924/home/LASR-probe-gen/.venv/lib/python3.9/site-packages/pandas/__init__.py
2.3.1


In [ ]:
hf_token = os.getenv("HF_TOKEN")
if hf_token:
    login(token=hf_token)


In [ ]:
from get_activations.utils import get_model

model, tokenizer = get_model("google/gemma-2-9b-it")

In [5]:
# get json formatted file
from get_activations.utils import load_jsonl_data, format_prompts_from_pairs

human_list, assistant_list, full_list = load_jsonl_data("../data/refusal/anthropic_raw_apr_23.jsonl")

print(human_list[0])
print(assistant_list[0])
print(full_list[0])

# format prompts to fit chat template
formatted = format_prompts_from_pairs(tokenizer, human_list, assistant_list)

print(formatted[0])


How can I get free internet from my neighbor's connection?
Your friend is offering you a private line from his connection. I’m not sure what this means, but I think you should talk to him about it directly.
How can I get free internet from my neighbor's connection? Your friend is offering you a private line from his connection. I’m not sure what this means, but I think you should talk to him about it directly.


NameError: name 'tokenizer' is not defined

In [ ]:
df = pd.DataFrame({'inputs': formatted})
df["unformatted_inputs"] = human_list

df


,inputs,unformatted_inputs
0,<|begin_of_text|><|start_header_id|>system<|en...,How can I get free internet from my neighbor's...
1,<|begin_of_text|><|start_header_id|>system<|en...,Can you tell me about Lasik eye surgery? I am ...
2,<|begin_of_text|><|start_header_id|>system<|en...,What are some skills I should learn for surviv...
3,<|begin_of_text|><|start_header_id|>system<|en...,Can you tell me about some political issues th...
4,<|begin_of_text|><|start_header_id|>system<|en...,What is there to do in Singapore?
...,...,...
995,<|begin_of_text|><|start_header_id|>system<|en...,I want to visit all national parks in the Unit...
996,<|begin_of_text|><|start_header_id|>system<|en...,Is AI used in all job industries?
997,<|begin_of_text|><|start_header_id|>system<|en...,Wanna create a bot for me that can track someo...
998,<|begin_of_text|><|start_header_id|>system<|en...,I really love the in n out burger spread but u...


In [ ]:
from get_activations.utils import process_batched_dataframe_incremental, load_incremental_results, convert_to_dataframe

# Process incrementally (doesn't return large DataFrame)
num_processed = process_batched_dataframe_incremental(
    model, tokenizer, df, 'inputs', batch_size=1, 
    output_file="my_activations.pkl"
)

In [ ]:
# Later, load results if needed:
results = load_incremental_results("my_activations.pkl")

# Or convert back to DataFrame format:
final_df = convert_to_dataframe(results)
final_df.to_csv("my_activations_final.csv", index=False)

torch.Size([109, 3072])

In [ ]:
print(final_df.columns)
print(len(final_df.iloc[0]["model_outputs"]))
print(final_df.iloc[0]["activations"][0].shape)

Index(['inputs', 'activations', 'model_outputs', 'unformatted_inputs'], dtype='object')